In [1]:
import sys,os,os.path
sys.path.append("/Users/user/Desktop/PhD/Repositories/IC/")
sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
if not os.environ.get("ICTDIR"):
    os.environ["ICTDIR"] = "/Users/user/Desktop/PhD/Repositories/IC"

import invisible_cities.core.core_functions as     coref
import invisible_cities.io  .dst_io         as     dstio
from invisible_cities.io.pmaps_io import load_pmaps
from invisible_cities.io.pmaps_io import load_pmaps_as_df
from invisible_cities.database.load_db  import DataPMT, DataSiPM


from matplotlib import rcParams
rcParams['mathtext.fontset'] = 'stix'
rcParams['font.family'] = 'STIXGeneral'
rcParams['figure.figsize'] = [8, 6]
rcParams['font.size'] = 15


import re
import glob
import numpy  as np
import pandas as pd
import tables as tb
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.optimize import curve_fit
import tables

from scipy.spatial import cKDTree
import matplotlib.patches as patches
import matplotlib.colors as mcolors
from   invisible_cities.database               import load_db

from invisible_cities.io.pmaps_io import load_pmaps
from invisible_cities.io.pmaps_io import load_pmaps_as_df

In [18]:
detector_info = load_db.DataSiPM('next100', 0)

In [ ]:
def median_std_method(wfs, nsigma=3):
    charges = np.sum(wfs, axis=1)
    threshold = np.median(charges) + nsigma * np.std(charges)
    return charges >= threshold


def hard_cut_method(wfs, threshold=5):
    charges = np.sum(wfs, axis=1)
    return charges >= threshold


def top_n_method(wfs, n=10):
    charges = np.sum(wfs, axis=1)
    idx = np.argsort(charges)[-n:]

    selected_ids = np.zeros_like(charges, dtype=bool)
    selected_ids[idx] = True
    return selected_ids

In [30]:
def kill_isolated_sipms(selected_ids, sipm_x, sipm_y, proximity_threshold):
    selected_ids_no_isolated = selected_ids.copy()

    for i in np.where(selected_ids)[0]:
        x, y = sipm_x[i], sipm_y[i]

        distances = np.sqrt((sipm_x - x)**2 + (sipm_y - y)**2)

        n_neighbors = np.sum((distances < proximity_threshold) & selected_ids)

        if n_neighbors <= 1:
            selected_ids_no_isolated[i] = False

    return selected_ids_no_isolated

In [31]:
def apply_radius_padding(selected_ids_no_isolated, sipm_x, sipm_y, radius_cut):
    sipm_ids_with_signal = np.zeros_like(selected_ids_no_isolated, dtype=bool)

    for i in np.where(selected_ids_no_isolated)[0]:
        x, y = sipm_x[i], sipm_y[i]
        distances = np.sqrt((sipm_x - x)**2 + (sipm_y - y)**2)
        sipm_ids_with_signal |= distances < radius_cut

    return sipm_ids_with_signal


In [32]:
def make_sipm_selection(wfs, selection_func, selection_kwargs, 
                        proximity_threshold, padding):
    """
    SiPM selection pipeline, applies SiPM cuts based on user input.
    A first selection of SiPMs is made, isolated SiPMs are removed
    and padding is added around the SiPMs that are left.

    Parameters
    ----------
    wfs : object
        Contains the waveforms of each SiPM.
    selection_func : callable
        Function returning a boolean mask of energetic SiPMs.
    selection_kwargs : dict
        Arguments passed to selection_func.
    proximity_threshold : float
        Threshold used to identify isolated SiPMs.
    padding : float
        Radial padding added to each SiPM that passes the selections.
    """

    sipm_x = detector_info.X
    sipm_y = detector_info.Y

    selected_ids = selection_func(wfs, **selection_kwargs)

    selected_ids_no_isolated = kill_isolated_sipms(
        selected_ids,
        sipm_x,
        sipm_y,
        proximity_threshold
    )

    sipm_ids_with_signal = apply_radius_padding(
        selected_ids_no_isolated,
        sipm_x,
        sipm_y,
        padding
    )

    return sipm_ids_with_signal
